In [1]:
%%capture
import json
from gensim.summarization import bm25
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm

# source : https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/summarization/bm25.py

In [2]:
with open('./datasets/train_set_aan.json') as f:
    train_set = json.load(f)
    
with open('./datasets/test_set_aan.json') as f:
    test_set = json.load(f)

In [3]:
corpus = [(ref['title'] + ' ' + ref['abstract']).split() for ref in train_set]
ids = [ref['id'] for ref in train_set]
model = bm25.BM25(corpus)

In [9]:
ground_truth = dict([(ref['id'], ref['references']) for ref in test_set])
recommendation = {}

for input_ref in tqdm(test_set):
    document = (input_ref['title'] + ' ' + input_ref['abstract']).split()
    candidate_scores = [(i, score) for i, score in enumerate(model.get_scores(document))]
        
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    
    recommendation[input_ref['id']] = [ids[cs[0]] for cs in candidate_scores[:100]]
    
len(ground_truth) - len(recommendation)

0

In [10]:
with open('./evaloffsets/base_bm25_gt.json', 'w') as f:
    json.dump(ground_truth, f)
    
with open('./evaloffsets/base_bm25_rec.json', 'w') as f:
    json.dump(recommendation, f)